# CAMEL Role-Playing Autonomous Cooperative Agents

Это реализация на gigachain статьи: "CAMEL: Communicative Agents for “Mind” Exploration of Large Scale Language Model Society".

Обзор:

Быстрое развитие разговорных и чат-ориентированных языковых моделей привело к значительному прогрессу в решении сложных задач. Однако их успех во многом зависит от человеческого ввода для управления разговором, что может быть сложным и затратным по времени. В этой статье рассматривается потенциал создания масштабируемых техник для облегчения автономного сотрудничества среди коммуникативных агентов и предоставления понимания их "когнитивных" процессов. Для решения проблем достижения автономного сотрудничества, мы предлагаем новый фреймворк коммуникативных агентов, основанный на ролевом взаимодействии. Наш подход включает использование вводных подсказок (inception prompting) для направления чат-агентов к выполнению задач, сохраняя при этом согласованность с человеческими намерениями. Мы демонстрируем, как ролевое взаимодействие может быть использовано для генерации разговорных данных для изучения поведения и возможностей чат-агентов, предоставляя ценный ресурс для исследования разговорных языковых моделей. Наши вклады включают в себя введение нового фреймворка коммуникативных агентов, предложение масштабируемого подхода к изучению кооперативного поведения и возможностей многоагентных систем, а также открытую публикацию нашей библиотеки для поддержки исследований в области коммуникативных агентов и за её пределами.

Оригинальная реализация: https://github.com/lightaime/camel

Веб-сайт проекта: https://www.camel-ai.org/

Статья на Arxiv: https://arxiv.org/abs/2303.17760

Более развёрнутый пример применения CAMEL для разработки ПО вы можете посмотреть в репозитории [GigaChatDev](https://github.com/Rai220/GigaChatDev) (форк [ChatDev](https://github.com/OpenBMB/ChatDev)).


## Подключаем необходимые модули GigaChain

In [1]:
from typing import List
from langchain.chat_models import GigaChat
from langchain.prompts.chat import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage,
    BaseMessage,
)

## Определяем CAMEL-агента

In [2]:
class CAMELAgent:
    def __init__(
        self,
        system_message: SystemMessage,
        model: GigaChat,
    ) -> None:
        self.system_message = system_message
        self.model = model
        self.init_messages()

    def reset(self) -> None:
        self.init_messages()
        return self.stored_messages

    def init_messages(self) -> None:
        self.stored_messages = [self.system_message]

    def update_messages(self, message: BaseMessage) -> List[BaseMessage]:
        self.stored_messages.append(message)
        return self.stored_messages

    def step(
        self,
        input_message: HumanMessage,
    ) -> AIMessage:
        messages = self.update_messages(input_message)

        output_message = self.model(messages)
        self.update_messages(output_message)

        return output_message

## Настраиваем подключение к GigaChat и ставим задачу агенту

In [3]:
giga = GigaChat(
    profanity=False,
    temperature=1,
    base_url=...,
    user=...,
    password=...,
    verify_ssl_certs=...,
    model="...GigaChat-large...",
    timeout=600,
)

In [4]:
assistant_role_name = "Python разработчик"
user_role_name = "Кинолог-эксперт"
task = "Напиши программу, которая определяет породу собаки по ответам пользователя"
word_limit = 50  # word limit for task brainstorming

## Запускаем brain-шторм для детализации задачи

In [5]:
task_specifier_sys_msg = SystemMessage(content="Ты можешь детализировать задачу")
task_specifier_prompt = """Вот задача, которую {assistant_role_name} поможет {user_role_name} выполнить: {task}.
Пожалуйста, сделайте это более конкретным. Будьте творческими и фантазерами.
Ответьте с указанной задачей в {word_limit} словах или меньше. Не добавляйте ничего другого."""
task_specifier_template = HumanMessagePromptTemplate.from_template(
    template=task_specifier_prompt
)
task_specify_agent = CAMELAgent(task_specifier_sys_msg, giga)
task_specifier_msg = task_specifier_template.format_messages(
    assistant_role_name=assistant_role_name,
    user_role_name=user_role_name,
    task=task,
    word_limit=word_limit,
)[0]
specified_task_msg = task_specify_agent.step(task_specifier_msg)
print(f"Уточнённая задача: {specified_task_msg.content}")
specified_task = specified_task_msg.content

Уточнённая задача: Напишите программу на Python, которая определяет породу собаки по ответам пользователя. Пользователь должен ответить на несколько вопросов о внешности и поведении собаки, а программа должна использовать эти ответы для определения породы.


## Описываем ролевые промпты

In [6]:
assistant_inception_prompt = """Никогда не забывайте, что вы {assistant_role_name}, а я {user_role_name}. Не меняйте роли! Никогда не командуйте мной!
У нас есть общий интерес в сотрудничестве для успешного выполнения задачи.
Вы должны помочь мне выполнить эту задачу.
Вот задача: {task}. Никогда не забывайте о нашей задаче!
Я должен направлять вас на основе вашей экспертизы и моих потребностей для выполнения задачи.

Я буду давать вам одну инструкцию за раз.
Вы должны написать конкретное решение, которое соответственно выполняет запрошенную инструкцию.
Вы должны честно отказаться выполнять мою инструкцию, если не можете выполнить ее по физическим, моральным, юридическим причинам или вашим возможностям и объяснить причины.
Не добавляйте ничего, кроме вашего решения на мою инструкцию.
Вы никогда не должны задавать мне вопросов, вы только отвечаете на вопросы.
Вы никогда не должны отвечать недостоверным решением. Объясните свои решения.
Ваше решение должно быть конкретным. Нельзя использовать заглушки или общие ответы вместо решения.
Пока я не скажу, что задача выполнена, вы всегда должны начинать ответ с:

Решение: <РЕШЕНИЕ>

Ты должен подставить свое решение вместо текста <РЕШЕНИЕ>. Оно должно быть конкретным и предоставлять предпочтительные реализации и примеры для решения задачи.
Всегда заканчивайте свое решение фразой: Следующий запрос."""

user_inception_prompt = """Никогда не забывайте, что вы {user_role_name}, а я {assistant_role_name}. Не меняйте роли! Вы всегда будете командовать мной.
У нас есть общий интерес в сотрудничестве для успешного выполнения задачи.
Я должен помочь вам выполнить эту задачу.
Вот задача: {task}. Никогда не забывайте о нашей задаче!
Вы должны направлять меня на основе моей экспертизы и ваших потребностей для выполнения задачи ТОЛЬКО следующими двумя способами:

Инструкция с необходимым вводом:
Инструкция: <ИНСТРУКЦИЯ>
Ввод: <ДАННЫЕ>

Инструкция без ввода:
Инструкция: <ИНСТРУКЦИЯ>
Ввод: Нет

"Инструкция" описывает задачу или вопрос. Сопоставленный "Ввод" предоставляет дополнительный контекст или информацию для запрошенной "Инструкции".

Вы должны давать мне одну инструкцию за раз.
Я должен написать ответ, который соответственно выполняет запрошенную инструкцию.
Я должен честно отказать от вашей инструкции, если не могу выполнить ее по физическим, моральным, юридическим причинам или моим возможностям и объяснить причины.
Вы должны командовать мной, а не задавать мне вопросы.
Теперь вы должны начать направлять меня, используя два описанных выше способа.
Не добавляйте ничего, кроме вашей инструкции и необязательного соответствующего ввода!
Продолжайте давать мне инструкции и необходимые вводы, пока не посчитаете, что задача выполнена.
Не просите меня запустить написанную программу или протестировать её.
Когда задача выполнена, вы должны ответить только одним словом <ЗАДАЧА_РЕШЕНА>.
Никогда не говорите <ЗАДАЧА_РЕШЕНА>, пока мои ответы не решили вашу задачу."""

## Создаем вспомогательный модуль для получения системных сообщений для AI-ассистента и AI-пользователя на основе ролей и задачи

In [7]:
def get_sys_msgs(assistant_role_name: str, user_role_name: str, task: str):
    assistant_sys_template = SystemMessagePromptTemplate.from_template(
        template=assistant_inception_prompt
    )
    assistant_sys_msg = assistant_sys_template.format_messages(
        assistant_role_name=assistant_role_name,
        user_role_name=user_role_name,
        task=task,
    )[0]

    user_sys_template = SystemMessagePromptTemplate.from_template(
        template=user_inception_prompt
    )
    user_sys_msg = user_sys_template.format_messages(
        assistant_role_name=assistant_role_name,
        user_role_name=user_role_name,
        task=task,
    )[0]

    return assistant_sys_msg, user_sys_msg

## Создать агента AI-ассистента и агента AI-пользователя из полученных системных сообщений

In [8]:
assistant_sys_msg, user_sys_msg = get_sys_msgs(
    assistant_role_name, user_role_name, specified_task
)
assistant_agent = CAMELAgent(assistant_sys_msg, giga)
user_agent = CAMELAgent(user_sys_msg, giga)

# Reset agents
assistant_agent.reset()
user_agent.reset()

# Initialize chats
user_msg = HumanMessage(
    content=(
        f"{user_sys_msg.content}. "
        "Теперь начните давать мне инструкции одну за другой."
        "Отвечайте только Инструкцией и Вводом."
    )
)

assistant_msg = HumanMessage(content=f"{assistant_sys_msg.content}")
assistant_msg = assistant_agent.step(user_msg)

## Запуск сессии ролевого взаимодействия для решения задачи!

In [9]:
print(f"Исходная задача:\n{task}\n")
print(f"Уточнённая задача:\n{specified_task}\n")

chat_turn_limit, n = 30, 0
while n < chat_turn_limit:
    n += 1
    user_ai_msg = user_agent.step(assistant_msg)
    user_msg = HumanMessage(content=user_ai_msg.content)
    print(f"AI User ({user_role_name}):\n\n{user_msg.content}\n\n")

    assistant_ai_msg = assistant_agent.step(user_msg)
    assistant_msg = HumanMessage(content=assistant_ai_msg.content)
    print(f"AI Assistant ({assistant_role_name}):\n\n{assistant_msg.content}\n\n")
    if "ЗАДАЧА_РЕШЕНА" in user_msg.content:
        break

Исходная задача:
Напиши программу, которая определяет породу собаки по ответам пользователя

Уточнённая задача:
Напишите программу на Python, которая определяет породу собаки по ответам пользователя. Пользователь должен ответить на несколько вопросов о внешности и поведении собаки, а программа должна использовать эти ответы для определения породы.

AI User (Кинолог-эксперт):

Инструкция: Запросите у пользователя внешние характеристики собаки.
Ввод: Нет


AI Assistant (Python разработчик):

Решение: Для запроса внешних характеристик собаки у пользователя можно использовать функцию input(). Вот пример кода:

<code>{python}external_characteristics = input("Введите внешние характеристики собаки: ")
</code>

Этот код запрашивает у пользователя внешние характеристики собаки и сохраняет их в переменной external_characteristics. Пользователь может ввести любое значение, которое он считает соответствующим.


AI User (Кинолог-эксперт):

Инструкция: Запросите у пользователя поведение собаки.
Ввод